# Create folder directories - skip if done

In [1]:
!ls ~/workspace/turb

data01_01  data03_01  data05_01  data07_01  data09_01  data11_01
data01_02  data03_02  data05_02  data07_02  data09_02  data11_02
data01_03  data03_03  data05_03  data07_03  data09_03  data11_03
data02_01  data04_01  data06_01  data08_01  data10_01  data12_01
data02_02  data04_02  data06_02  data08_02  data10_02  data12_02
data02_03  data04_03  data06_03  data08_03  data10_03  data12_03


In [2]:
%cd ~/workspace/turb/data01_01

/home/idies/workspace/turb/data01_01


In [3]:
# creatuple list of folder on filedb to which data should be copied
# you may want to change this to avoid certain volumes
folders=[f'/home/idies/workspace/turb/data{str(f).zfill(2)}_{str(d).zfill(2)}/zarr/ncar-zarr/'  for f in range(1,13) for d in range(1,4)]
folders[:5]

['/home/idies/workspace/turb/data01_01/zarr/ncar-zarr/',
 '/home/idies/workspace/turb/data01_02/zarr/ncar-zarr/',
 '/home/idies/workspace/turb/data01_03/zarr/ncar-zarr/',
 '/home/idies/workspace/turb/data02_01/zarr/ncar-zarr/',
 '/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/']

In [4]:
# Avoiding 7-2 and 9-2 - they're too full as of May 2023
# folders.remove("/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data09_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/ncar-zarr/") # This is already created

In [5]:
# Chatgpt - create folders above - skip if already created

import os

def create_folders(paths):
    for path in paths:
        os.makedirs(path, exist_ok=True)
        print(f"Folder created at {path}")

# create_folders(folders)


In [6]:
# create dataXX_XX/zarr/ncar-zarr/timestep
timesteps = 6

timestep_folders = [f + "00" + str(i) for i in range(timesteps) for f in folders]
timestep_folders

['/home/idies/workspace/turb/data01_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data01_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data01_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data02_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data02_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data03_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data03_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data03_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data04_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data04_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data04_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data05_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data05_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data05_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data06_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data06_02/z

In [7]:
# Create subfolders - skip if done already

# create_folders(timestep_folders)

In [17]:
variable_names_set = {"e", "t", "p", "u", "v", "w"}

dest_directory_dict = {}

for index, value in enumerate(variable_names_set):
    dest_directory_dict[value] = [t + "/" + value for t in timestep_folders]

dest_directory_dict

{'w': ['/home/idies/workspace/turb/data01_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data01_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data01_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data02_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data02_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data03_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data03_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data03_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data04_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data04_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data04_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data05_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data05_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data05_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data06_01/zarr/ncar-

In [10]:
# create dataXX_XX/zarr/ncar-zarr/timestep/Variable_name
# create_folders(variable_folders)

# Write to folders

In [ ]:
# Edited from ChatGPT
import os
import shutil

source_directory = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr'
destination_directory = '/home/idies/workspace/turb/'

disk_prefix = 'data'
num_disks_per_path = 3
num_paths = 12

# Generate the list of destination directories
destination_directories = [f'{disk_prefix}{i:02d}_{j:02d}' for i in range(1, num_paths+1) for j in range(1, num_disks_per_path+1)]
num_disks = len(destination_directories)

# Traverse the source directory and distribute files to destination directories
for root, dirs, files in os.walk(source_directory):
    for file in files:
        source_path = os.path.join(root, file)

        # Determine the destination disk
        disk_index = sum([ord(c) for c in file]) % num_disks
        destination_disk = os.path.join(destination_directory, destination_directories[disk_index])

        # Create the corresponding directory structure on the destination disk
        destination_path = os.path.join(destination_disk, os.path.relpath(root, source_directory))
        os.makedirs(destination_path, exist_ok=True)

        # Copy the file to the destination disk
        shutil.copy(source_path, destination_path)


In [13]:
%cd /home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/e

/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/e


In [15]:
file_list = [file for file in os.listdir() if not file.startswith('.')]

In [16]:
len(file_list)

4096

In [12]:
i = 0

source_directory = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/'
timestep_prefix = "jhd-00{t}-nc-zarr"

# [timestep_prefix.format(t=t) for t in range(timesteps)]

for t in range(timesteps):
    timestep_dir = timestep_prefix.format(t=t)
    
    print("Working on " + timestep_dir)
    
    for index, varname in enumerate(variable_names_set):
        pwd = source_directory + timestep_dir + "/" + varname
        print("Current working dir: ", pwd)
        
        file_list = [file for file in os.listdir() if not file.startswith('.')] # Don't move zarr metadata. TODO deal with this later
        
        i += 1

Working on jhd-000-nc-zarr
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/w
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/t
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/p
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/u
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/e
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/v
Working on jhd-001-nc-zarr
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-001-nc-zarr/w
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-001-nc-zarr/t
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-001-nc-zarr/p
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-001-nc-zarr/u
/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-001-nc-zarr/e
/home/idies/workspace/turb/data02_02/ariel-6

# TODO Fix Zarr Metadata

In [10]:
z = zarr.open("/home/idies/workspace/turb/data02_03/zarr/ncar-zarr")
z.info

Name,/
Type,zarr.hierarchy.Group
Read-only,False
Store type,zarr.storage.DirectoryStore
No. members,0
No. arrays,0
No. groups,0


In [11]:
zarr.consolidate_metadata("/home/idies/workspace/turb/data02_03/zarr/ncar-zarr/e")

PathNotFoundError: nothing found at path ''